# ML testing: experiment #3 a- unharmonized

This notebook involves testing for the MRI harmonization research. This notebook shows mixed_dataset (TOP + StrokeMRI) based models without harmonization, applied to variosu datasets

Data: StrokeMRI, TOP, SABRE, Insight46, EDIS, (HELIUS pending)

Harmonisation: None

Training data: Norment whichh is StrokeMRI and TOP togehter

Testing data: test set from Norment

Futher data applied to: SABRE, Insight46, EDIS, (HELIUS pending)

Validation method: K-fold, double-stratified

Brain-age algorithms: LR, lasso, extra trees, elasticCV net fully tested (but not optimized parameters), additionals partly

Outputs: ----

### import libraries

In [ ]:
import os       # using operating system dependent functionality (folders)
import sys

import glob
from functools import reduce

import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import matplotlib.pyplot as plt

# # demo stuff
import ipywidgets as widgets
from ipywidgets import interactive
# import seaborn 

# ml stuff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
#from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn import tree
from sklearn import metrics
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score

import joblib

sys.path.insert(0, '../') # path to functions
import cvasl.seperated as sep

### import data

In [ ]:
# Datasets for this work
EDIS_path = 'our_datasets_healed/EDIS/'
HELIUS_path = 'our_datasets_healed/HELIUS/'
Insight46_path = 'our_datasets_healed/Insight46/'
SABRE_path = 'our_datasets_healed/SABRE/'
MRI_path = 'our_datasets_healed/StrokeMRI/'
TOP_path = 'our_datasets_healed/TOP/'
file_name = 'TrainingDataComplete.csv'

TOP_file = os.path.join(TOP_path, file_name)
MRI_file = os.path.join(MRI_path, file_name)
EDIS_file = os.path.join(EDIS_path, file_name)
HELIUS_file = os.path.join(HELIUS_path, file_name)
Insight46_file = os.path.join(Insight46_path, file_name)
SABRE_file = os.path.join(SABRE_path, file_name)

EDIS_n = pd.read_csv(EDIS_file, index_col=0)
HELIUS_n = pd.read_csv(HELIUS_file, index_col=0)
Insight46_n = pd.read_csv(Insight46_file, index_col=0)
SABRE_n = pd.read_csv(SABRE_file, index_col=0)
TOP_n = pd.read_csv(TOP_file, index_col=0)
MRI_n = pd.read_csv(MRI_file, index_col=0)

In [ ]:
MRI_n.columns

In [ ]:
# SABRE =SABRE.dropna()
# #SABRE.isna().sum()

In [ ]:
HELIUS = HELIUS_n.drop(['ID', 'Site'], axis=1)
Insight46 = Insight46_n.drop(['ID', 'Site'], axis=1)
EDIS = EDIS_n.drop(['ID', 'Site'], axis=1)
SABRE = SABRE_n.drop(['ID', 'Site'], axis=1)
StrokeMRI = MRI_n.drop(['ID', 'Site'], axis=1)

In [ ]:
TOP = TOP_n.drop(['ID', 'Site'], axis=1)
TOP.head(2)

In [ ]:
TOP.columns

In [ ]:
HELIUS.head(3)

In [ ]:
TOP.columns = TOP.columns.str.lower() 
StrokeMRI.columns = StrokeMRI.columns.str.lower() 
Insight46.columns = Insight46.columns.str.lower() 
EDIS.columns = EDIS.columns.str.lower() 
SABRE.columns= SABRE.columns.str.lower() 
HELIUS.columns= HELIUS.columns.str.lower() 

In [ ]:
EDIS.columns

In [ ]:
datasets = [EDIS, SABRE, Insight46, TOP, StrokeMRI, HELIUS]
for qset in datasets:
    print(qset.sex.unique())

In [ ]:
sex_mapping = {1:0,2:1}
Insight46 = Insight46.assign(sex = Insight46.sex.map(sex_mapping))
EDIS =EDIS.assign(sex = EDIS.sex.map(sex_mapping))
SABRE=SABRE.assign(sex = SABRE.sex.map(sex_mapping))

In [ ]:
# check and rename known repeater in HELIUS and SABRE
sabres = set(SABRE.participant_id)
heliar = set(HELIUS.participant_id)
x = sabres.intersection(heliar)
print(x)

In [ ]:
# # check for any duplicated patients between stroke and mri
# strokers = set(StrokeMRI.participant_id)
# topers = set(TOP.participant_id)
# z = strokers.intersection(topers)
# print(z) 

In [ ]:
HELIUS.loc[HELIUS['participant_id']=='sub-153852_1', 'participant_id'] = 'sub-153852_1H'

In [ ]:
sabres = set(SABRE.participant_id)
heliar = set(HELIUS.participant_id)
x = sabres.intersection(heliar)
print(x)

In [ ]:
# make mixed StrokeMRI and TOP dataset
mixed_data = pd.concat([TOP, StrokeMRI], sort=False)

In [ ]:
#mixed_data.tail(3)

In [ ]:
#give a checkbox for out put folder
loged_feat = widgets.ToggleButton(
    value=False,
    description='Click me if some features logged',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
loged_feat

In [ ]:
loged_feat.value

In [ ]:
if loged_feat.value == False:
    output_folder = '3a_no_log_outputs'
else:
    output_folder = '3a_loged_outputs'

os.makedirs(output_folder, exist_ok=True)

## Build ML models

In [ ]:
ml_matrix = mixed_data.drop('participant_id', axis=1)
X = ml_matrix.drop('age', axis =1)
X = X.values
y = ml_matrix['age'].values
y=y.astype('float')

In [ ]:
linr_k_frame, linr_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('linear regression', 'unharm_mix_linr', LinearRegression(), ml_matrix, X[:,1:], y)

In [ ]:
linr_k_frame

In [ ]:
linr_k_frame.to_csv(output_folder + '/unharmonized_linr_k_frame.csv')

In [ ]:
avg_linr = sep.avg_k_folds(linr_k_frame)
avg_linr

In [ ]:
linr_y_frame

In [ ]:
linr_y_frame.to_csv(output_folder + '/unharmonized_linr_y_frame.csv')

In [ ]:
linr = models[0]
linr[0]

In [ ]:
# check if model folder exists and if not , then create
model_folder = '../result_models/'
if not os.path.exists(model_folder):
    os.makedirs(model_folder)

In [ ]:
## optional save models
#joblib.dump(linr[0], ('../result_models/'+ 'unharm_mix_linr0.sav'))
#joblib.dump(linr[1], ('../result_models/'+ 'unharm_mix_linr1.sav'))
#joblib.dump(linr[2], ('../result_models/'+ 'unharm_mix_linr2.sav'))
#joblib.dump(linr[3], ('../result_models/'+ 'unharm_mix_linr3.sav'))
#joblib.dump(linr[4], ('../result_models/'+ 'unharm_mix_linr4.sav'))

In [ ]:
llreg_k_frame, llreg_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('lasso regression', 'unharm_mix_llreg',  linear_model.LassoLars(alpha=0.01), ml_matrix, X[:,1:], y)
llreg_k_frame

In [ ]:
llreg_k_frame.to_csv(output_folder + '/unharmonized_llreg_k_frame.csv')

In [ ]:
avg_llreg = sep.avg_k_folds(llreg_k_frame)
avg_llreg

In [ ]:
llreg_y_frame

In [ ]:
llreg_y_frame.to_csv(output_folder + '/unharmonized_llreg_y_frame.csv')

In [ ]:
llreg = models[0]
llreg[0]

In [ ]:
## optional save models
#joblib.dump(llreg[0], ('../result_models/'+ 'unharm_mix_llreg0.sav'))
#joblib.dump(llreg[1], ('../result_models/'+ 'unharm_mix_llreg1.sav'))
#joblib.dump(llreg[2], ('../result_models/'+ 'unharm_mix_llreg2.sav'))
#joblib.dump(llreg[3], ('../result_models/'+ 'unharm_mix_llreg3.sav'))
#joblib.dump(llreg[4], ('../result_models/'+ 'unharm_mix_llreg4.sav'))

In [ ]:
dtree_k_frame, dtree_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('decision tree', 'unharm_mix_dtree',  tree.DecisionTreeRegressor(), ml_matrix, X[:,1:], y)
dtree_k_frame

In [ ]:
dtree_k_frame.to_csv(output_folder + '/unharmonized_dtree_k_frame.csv')

In [ ]:
avg_dtree = sep.avg_k_folds(dtree_k_frame)
avg_dtree

In [ ]:
dtree_y_frame

In [ ]:
dtree_y_frame.to_csv(output_folder + '/unharmonized_dtree_y_frame.csv')

In [ ]:
dtree = models[0]
dtree[0]

In [ ]:
regr_k_frame, regr_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('MLP regression', 'unharm_mix_regr',   MLPRegressor(random_state=1, max_iter=700), ml_matrix, X[:,1:], y)
regr_k_frame

In [ ]:
regr_k_frame.to_csv(output_folder + '/unharmonized_regr_k_frame.csv')

In [ ]:
avg_regr = sep.avg_k_folds(regr_k_frame)
avg_regr

In [ ]:
regr_y_frame

In [ ]:
regr_y_frame.to_csv(output_folder + '/unharmonized_regr_y_frame.csv')

In [ ]:
regr = models[0]
regr[0]

In [ ]:
svrp2_k_frame, svrp2_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('support vector reg poly2', 'unharm_mix_svrp2',   SVR(C=1.0, kernel='poly', degree =2, epsilon=0.2), ml_matrix, X[:,1:], y)
svrp2_k_frame

In [ ]:
avg_svrp2 = sep.avg_k_folds(svrp2_k_frame)
avg_svrp2

In [ ]:
svrp2_y_frame

In [ ]:
svrp2 = models[0]
svrp2[0]

In [ ]:
eregr_k_frame, eregr_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('elasticnetCV', 'unharm_mix_eregr',  ElasticNetCV(cv=5, random_state=12), ml_matrix, X[:,1:], y)
eregr_k_frame

In [ ]:
eregr_k_frame.to_csv(output_folder + '/unharmonized_eregr_k_frame.csv')

In [ ]:
avg_eregr = sep.avg_k_folds(eregr_k_frame)
avg_eregr

In [ ]:
eregr_y_frame

In [ ]:
eregr = models[0]
eregr[0]

In [ ]:
etreg_k_frame, etreg_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('extra trees', 'unharm_mix_etreg',  ExtraTreesRegressor(n_estimators=100, random_state=0), ml_matrix, X[:,1:], y)
etreg_k_frame

In [ ]:
etreg_k_frame.to_csv(output_folder + '/unharomized_etreg_k_frame.csv')

In [ ]:
avg_etreg = sep.avg_k_folds(etreg_k_frame)
avg_etreg

In [ ]:
etreg_y_frame

In [ ]:
etreg_y_frame.to_csv(output_folder + '/unharmonized_etreg_y_frame.csv')

In [ ]:
etreg = models[0]
etreg[0]

In [ ]:
## optional save models
#joblib.dump(etreg[0], ('../result_models/'+ 'unharm_mix_etreg0.sav'))
#joblib.dump(etreg[1], ('../result_models/'+ 'unharm_mix_etreg1.sav'))
#joblib.dump(etreg[2], ('../result_models/'+ 'unharm_mix_etreg2.sav'))
#joblib.dump(etreg[3], ('../result_models/'+ 'unharm_mix_etreg3.sav'))
#joblib.dump(etreg[4], ('../result_models/'+ 'unharm_mix_etreg4.sav'))

Show results ON AVERAGE for each model

In [ ]:
mixed_based_unharmonized_on_testmix =pd.concat([avg_linr,
                   avg_llreg,
                   avg_dtree,
                   avg_regr,
                   avg_svrp2,
                   avg_eregr,
                   avg_etreg],
                  axis=0)
mixed_based_unharmonized_on_testmix

In [ ]:
mixed_based_unharmonized_on_testmix.to_csv(output_folder + '/mixed_based_unharmonized_on_testmix_AVERAGES.csv')

## Run mixed models on other datasets (All TOP- StrokeMRI blended model on INsight and Sabre)
now we will make some models based on all top an strokeMRI data together

In [ ]:
TOPMRI = pd.concat([TOP, StrokeMRI])
TOPMRI.head(3)

In [ ]:
ml_matrix = TOPMRI.drop('participant_id', axis=1)
X = ml_matrix.drop('age', axis =1)
X = X.values
X = X.astype('float')
y = ml_matrix['age'].values
y=y.astype('float')


In [ ]:
# now we decide to do no test train split, rather take all
X_train = X
y_train = y

In [ ]:
MIXlinr = LinearRegression()
MIXlinr.fit(X_train, y_train)

In [ ]:
MIXllreg = linear_model.LassoLars(alpha=0.01)
MIXllreg.fit(X_train, y_train)

In [ ]:
MIXeregr = ElasticNetCV(cv=5, random_state=17)
MIXeregr.fit(X_train, y_train)


In [ ]:
MIXetreg = ExtraTreesRegressor(n_estimators=100, random_state=0)
MIXetreg.fit(X_train, y_train)

In [ ]:
##  Save these four best models

In [ ]:
## optional save models
#joblib.dump(MIXlinr, ('../result_models/'+ 'unharm_mix_MIXlinr.sav'))
#joblib.dump(MIXllreg, ('../result_models/'+ 'unharm_mix_MIXllreg1.sav'))
#joblib.dump(MIXeregr, ('../result_models/'+ 'unharm_mix_MIXeregr3.sav'))
#joblib.dump(MIXetreg, ('../result_models/'+ 'unharm_mix_MIXetreg4.sav'))

# Running mixed model over SABRE dataset

In [ ]:
# linr, llreg,eregr, etreg

In [ ]:
sabre_ml_matrix = SABRE.drop('participant_id', axis=1)
X_sabre = sabre_ml_matrix.drop('age', axis =1)
#X_sabre.shape

#X_sabre.columns

X_sabre = X_sabre.values
X_sabre = X_sabre.astype('float')
y_sabre = sabre_ml_matrix['age'].values
y_sabre=y_sabre.astype('float')

In [ ]:
X_sabre_test = X_sabre
y_sabre_test = y_sabre

In [ ]:
y_sabre_pred = MIXlinr.predict(X_sabre_test)

In [ ]:
data= [[
    'Linear Reg',
    'unharm_mix_MIXlinr.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    MIXlinr.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
linr_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
linr_results_sabre

In [ ]:
data = {'real': y_sabre_test, 'predicted': y_sabre_pred}
y_frame_linr_sabre = pd.DataFrame(data)
y_frame_linr_sabre

In [ ]:
y_frame_linr_sabre.to_csv(output_folder + '/unharm_y_frame_linr_sabre_MIX.csv')

In [ ]:
y_sabre_pred = MIXllreg.predict(X_sabre_test)

In [ ]:
data= [[
    'Lasso',
    'unharm_mix_MIXlassor.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    MIXllreg.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
llreg_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
llreg_results_sabre

In [ ]:
data = {'real': y_sabre_test, 'predicted': y_sabre_pred}
y_frame_llreg_sabre = pd.DataFrame(data)
y_frame_llreg_sabre

In [ ]:
y_frame_llreg_sabre.to_csv(output_folder + '/unharm_y_frame_llreg_sabre_MIX.csv')

In [ ]:
y_sabre_pred = MIXeregr.predict(X_sabre_test)

In [ ]:
data= [[
    'ElasticnetCV',
    'unharm_mri_elasticregr.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    MIXeregr.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
eregr_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
eregr_results_sabre

In [ ]:
data = {'real': y_sabre_test, 'predicted': y_sabre_pred}
y_frame_eregr_sabre = pd.DataFrame(data)
y_frame_eregr_sabre

In [ ]:
y_frame_eregr_sabre.to_csv(output_folder + '/unharm_y_frame_eregr_sabre_MIX.csv')

In [ ]:
y_sabre_pred = MIXetreg.predict(X_sabre_test)

In [ ]:
data= [[
    'Extra trees',
    'unharm_mri_etreg.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    MIXetreg.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
etreg_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
etreg_results_sabre

In [ ]:
data = {'real': y_sabre_test, 'predicted': y_sabre_pred}
y_frame_etregr_sabre = pd.DataFrame(data)
y_frame_etregr_sabre

In [ ]:
y_frame_etregr_sabre.to_csv(output_folder + '/y_frame_etregr_sabre_MIX.csv')

In [ ]:
mix_based_unharmonized_on_sabre =pd.concat([linr_results_sabre,
                   llreg_results_sabre,
                   eregr_results_sabre,
                  etreg_results_sabre],
                  axis=0)
mix_based_unharmonized_on_sabre

In [ ]:
mix_based_unharmonized_on_sabre.to_csv(output_folder + '/mix_based_unharmonized_on_sabre.csv')

# Running mixed model over Insight46 dataset

In [ ]:
insight_ml_matrix = Insight46.drop('participant_id', axis=1)
X_insight = insight_ml_matrix.drop('age', axis =1)
X_insight = X_insight.values
X_insight = X_insight.astype('float')
y_insight = insight_ml_matrix['age'].values
y_insight= y_insight.astype('float')

In [ ]:
X_insight_test = X_insight
y_insight_test = y_insight

In [ ]:
y_insight_pred = MIXlinr.predict(X_insight_test)

In [ ]:
data= [[
    'Linear Reg',
    'unharm_mix_linr0.sav',
    mean_absolute_error(y_insight_test, y_insight_pred),
    MIXlinr.score(X_insight_test,y_insight_test),
    metrics.explained_variance_score(y_insight_test, y_insight_pred)]]
linr_results_insight = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
linr_results_insight

In [ ]:
data = {'real': y_insight_test, 'predicted': y_insight_pred}
y_frame_linr_insight = pd.DataFrame(data)
y_frame_linr_insight

In [ ]:
y_frame_linr_insight.to_csv(output_folder + '/y_frame_linr_insight_MIX.csv')

In [ ]:
y_insight_pred = MIXllreg.predict(X_insight_test)

In [ ]:
data= [[
    'Lasso',
    'unharm_mix_lassor0.sav',
    mean_absolute_error(y_insight_test, y_insight_pred),
    MIXllreg.score(X_insight_test,y_insight_test),
    metrics.explained_variance_score(y_insight_test, y_insight_pred)]]
llreg_results_insight = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
llreg_results_insight

In [ ]:
data = {'real': y_insight_test, 'predicted': y_insight_pred}
y_frame_llreg_insight = pd.DataFrame(data)
y_frame_llreg_insight

In [ ]:
y_frame_llreg_insight.to_csv(output_folder + '/y_frame_llreg_insight_MIX.csv')

In [ ]:
y_insight_pred = MIXeregr.predict(X_insight_test)

In [ ]:
data= [[
    'ElasticnetCV',
    'unharm_mri_elasticregr.sav',
    mean_absolute_error(y_insight_test, y_insight_pred),
    MIXeregr.score(X_insight_test,y_insight_test),
    metrics.explained_variance_score(y_insight_test, y_insight_pred)]]
eregr_results_insight = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
eregr_results_insight

In [ ]:
data = {'real': y_insight_test, 'predicted': y_insight_pred}
y_frame_eregr_insight = pd.DataFrame(data)
y_frame_eregr_insight

In [ ]:
y_frame_eregr_insight.to_csv(output_folder + '/y_frame_eregr_insight_MIX.csv')

In [ ]:
y_insight_pred = MIXetreg.predict(X_insight_test)

In [ ]:
data= [[
    'Extra trees',
    'unharm_mri_etreg.sav',
    mean_absolute_error(y_insight_test, y_insight_pred),
    MIXetreg.score(X_insight_test,y_insight_test),
    metrics.explained_variance_score(y_insight_test, y_insight_pred)]]
etreg_results_insight = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
etreg_results_insight

In [ ]:
data = {'real': y_insight_test, 'predicted': y_insight_pred}
y_frame_etregr_insight = pd.DataFrame(data)
y_frame_etregr_insight

In [ ]:
y_frame_etregr_insight.to_csv(output_folder + '/y_frame_etregr_insight_MIX.csv')

In [ ]:
mix_based_unharmonized_on_insight =pd.concat([linr_results_insight,
                   llreg_results_insight,
                   eregr_results_insight,
                   etreg_results_insight],
                  axis=0)
mix_based_unharmonized_on_insight

In [ ]:
mix_based_unharmonized_on_insight.to_csv(output_folder + '/mix_based_unharmonized_on_insight.csv')

# Run mixed model on EDIS

In [ ]:
edis_ml_matrix = EDIS.drop('participant_id', axis=1)
X_edis = edis_ml_matrix.drop('age', axis =1)
X_edis = X_edis.values
X_edis = X_edis.astype('float')
y_edis = edis_ml_matrix['age'].values
y_edis=y_edis.astype('float')

In [ ]:
X_edis_test = X_edis
y_edis_test = y_edis

In [ ]:
y_edis_pred = MIXlinr.predict(X_edis_test)

In [ ]:
data= [[
    'Linear Reg',
    'unharm_mix_MIXlinr.sav',
    mean_absolute_error(y_edis_test, y_edis_pred),
    MIXlinr.score(X_edis_test,y_edis_test),
    metrics.explained_variance_score(y_edis_test, y_edis_pred)]]
linr_results_edis = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
linr_results_edis

In [ ]:
data = {'real': y_edis_test, 'predicted': y_edis_pred}
y_frame_linr_edis = pd.DataFrame(data)
y_frame_linr_edis

In [ ]:
y_frame_linr_edis.to_csv(output_folder + '/unharm_y_frame_linr_edis_MIX.csv')

In [ ]:
y_edis_pred = MIXllreg.predict(X_edis_test)

In [ ]:
data= [[
    'Lasso',
    'unharm_mix_MIXlassor.sav',
    mean_absolute_error(y_edis_test, y_edis_pred),
    MIXllreg.score(X_edis_test,y_edis_test),
    metrics.explained_variance_score(y_edis_test, y_edis_pred)]]
llreg_results_edis = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
llreg_results_edis

In [ ]:
data = {'real': y_edis_test, 'predicted': y_edis_pred}
y_frame_llreg_edis = pd.DataFrame(data)
y_frame_llreg_edis

In [ ]:
y_frame_llreg_edis.to_csv(output_folder + '/unharm_y_frame_llreg_edis_MIX.csv')

In [ ]:
y_edis_pred = MIXeregr.predict(X_edis_test)

In [ ]:
data= [[
    'ElasticnetCV',
    'unharm_mri_elasticregr.sav',
    mean_absolute_error(y_edis_test, y_edis_pred),
    MIXeregr.score(X_edis_test,y_edis_test),
    metrics.explained_variance_score(y_edis_test, y_edis_pred)]]
eregr_results_edis = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
eregr_results_edis

In [ ]:
data = {'real': y_edis_test, 'predicted': y_edis_pred}
y_frame_eregr_edis = pd.DataFrame(data)
y_frame_eregr_edis

In [ ]:
y_frame_eregr_edis.to_csv(output_folder + '/y_frame_eregr_edis_MIX.csv')

In [ ]:
y_edis_pred = MIXetreg.predict(X_edis_test)

In [ ]:
data= [[
    'Extra trees',
    'unharm_mri_etreg.sav',
    mean_absolute_error(y_edis_test, y_edis_pred),
    MIXetreg.score(X_edis_test,y_edis_test),
    metrics.explained_variance_score(y_edis_test, y_edis_pred)]]
etreg_results_edis = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
etreg_results_edis

In [ ]:
data = {'real': y_edis_test, 'predicted': y_edis_pred}
y_frame_etregr_edis = pd.DataFrame(data)
y_frame_etregr_edis

In [ ]:
y_frame_etregr_edis.to_csv(output_folder + '/y_frame_etregr_edis_MIX.csv')

In [ ]:
mix_based_unharmonized_on_edis =pd.concat([linr_results_edis,
                   llreg_results_edis,
                   eregr_results_edis,
                  etreg_results_edis],
                  axis=0)
mix_based_unharmonized_on_edis

In [ ]:
mix_based_unharmonized_on_edis.to_csv(output_folder + '/mix_based_unharmonized_on_edis.csv')

# Run mixed model on HELIUS

In [ ]:
helius_ml_matrix = HELIUS.drop('participant_id', axis=1)
X_helius = helius_ml_matrix.drop('age', axis =1)
X_helius = X_helius.values
X_helius = X_helius.astype('float')
y_helius = helius_ml_matrix['age'].values
y_helius=y_helius.astype('float')

In [ ]:
X_helius_test = X_helius
y_helius_test = y_helius

In [ ]:
y_helius_pred = MIXlinr.predict(X_helius_test)

In [ ]:
data= [[
    'Linear Reg',
    'unharm_mix_MIXlinr_helius.sav',
    mean_absolute_error(y_helius_test, y_helius_pred),
    MIXlinr.score(X_helius_test,y_helius_test),
    metrics.explained_variance_score(y_helius_test, y_helius_pred)]]
linr_results_helius = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
linr_results_helius

In [ ]:
data = {'real': y_helius_test, 'predicted': y_helius_pred}
y_frame_linr_helius = pd.DataFrame(data)
y_frame_linr_helius

In [ ]:
y_frame_linr_helius.to_csv(output_folder + '/unharm_y_frame_linr_heliuss_MIX.csv')

In [ ]:
y_helius_pred = MIXllreg.predict(X_helius_test)

In [ ]:
data= [[
    'Lasso',
    'unharm_mix_MIXlassor_helius.sav',
    mean_absolute_error(y_helius_test, y_helius_pred),
    MIXllreg.score(X_helius_test,y_helius_test),
    metrics.explained_variance_score(y_helius_test, y_helius_pred)]]
llreg_results_helius = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
llreg_results_helius

In [ ]:
data = {'real': y_helius_test, 'predicted': y_helius_pred}
y_frame_llreg_helius = pd.DataFrame(data)
y_frame_llreg_helius

In [ ]:
y_frame_llreg_helius.to_csv(output_folder + '/unharm_y_frame_llreg_helius_MIX.csv')

In [ ]:
y_helius_pred = MIXeregr.predict(X_helius_test)

In [ ]:
data= [[
    'ElasticnetCV',
    'unharm_mri_elasticregr.sav',
    mean_absolute_error(y_helius_test, y_helius_pred),
    MIXeregr.score(X_helius_test,y_helius_test),
    metrics.explained_variance_score(y_helius_test, y_helius_pred)]]
eregr_results_helius = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
eregr_results_helius

In [ ]:
data = {'real': y_helius_test, 'predicted': y_helius_pred}
y_frame_eregr_helius = pd.DataFrame(data)
y_frame_eregr_helius

In [ ]:
y_frame_eregr_helius.to_csv(output_folder + '/y_frame_eregr_helius_MIX.csv')

In [ ]:
y_helius_pred = MIXetreg.predict(X_helius_test)

In [ ]:
data= [[
    'Extra trees',
    'unharm_mri_etreg.sav',
    mean_absolute_error(y_helius_test, y_helius_pred),
    MIXetreg.score(X_helius_test,y_helius_test),
    metrics.explained_variance_score(y_helius_test, y_helius_pred)]]
etreg_results_helius = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
etreg_results_helius

In [ ]:
data = {'real': y_helius_test, 'predicted': y_helius_pred}
y_frame_etregr_helius = pd.DataFrame(data)
y_frame_etregr_helius

In [ ]:
y_frame_etregr_helius.to_csv(output_folder + '/y_frame_etregr_helius_MIX.csv')

In [ ]:
mix_based_unharmonized_on_helius =pd.concat([linr_results_helius,
                   llreg_results_helius,
                   eregr_results_helius,
                  etreg_results_helius],
                  axis=0)
mix_based_unharmonized_on_helius

In [ ]:
mix_based_unharmonized_on_helius.to_csv(output_folder + '/mix_based_unharmonized_on_helius.csv')